In [1]:
import torch
import torch.nn as nn

import os
import json
from tqdm import tqdm

import src.models as mdls
import src.utils as utils

# Face recognition imports
from src.models import Triplet
from src.models import ConvEmbedding

from src.utils import TripletLoss


#downloading datasets
config = utils.get_options()


if config['use_colab']:
    utils.colab()
else:
    utils.check_if_datasets_are_downloaded()


import src.dataloaders as dataloaders

from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

Moving kaggle file

all datasets are in place


In [2]:
img_size = config['img_size']
# Define a directory to save the checkpoints
checkpoint_dir = "checkpoints/"

# Ensure the directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

#define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
#Face detection setup
detection_model = mdls.InspectorGadjet()
dataloader_for_detection = dataloaders.detection_dataloader

# loss_fn = nn.MSELoss()
loss_fn = mdls.combined_loss
optimizer = torch.optim.Adam(detection_model.parameters(), lr=1e-3)
detection_model.to(device)
# loss_fn.to(device)
None

In [ ]:
### Function to check how albumentation works
# batch = next(iter(dataloader_for_detection))
# utils.plot_images_with_bboxes(batch)

In [4]:
det_epochs = config['detection_epochs']
det_logging = config['detection_logging']
log_interval = config["detection_log_wieghts_interval"]
test_img_path = ''

if det_logging: 
    # read secrets for cometml logging
    with open('secrets.json') as secrets_file:
        secrets = json.load(secrets_file)

    # init experimenxt
    experiment = Experiment(
        api_key=secrets["api_key"],
        project_name=secrets["project_name"],
        workspace="reu-ds-club", 
        tags=["detection"],
    )

    hyper_params = {
        "model_name": config["model"],
        "use_colab": config['use_colab'], 
        "epochs": det_epochs,
        "batch_size": config['batch_size'], 
        "image_size": config['img_size'], 
    }

    experiment.log_parameters(hyper_params)

for epoch in range(det_epochs):
    epoch_loss = 0.0
    for sample in (pbar := tqdm(dataloader_for_detection)):

        img, box = sample[0].to(device), sample[1].to(device)
        img = img.to(torch.float32)

        box = box.to(torch.float32)
        optimizer.zero_grad()
        pred = detection_model(img)
        # loss = loss_fn(pred, box)
        

        loss = loss_fn(pred[0], pred[1], box)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    utils.save_img_after_epoch(test_img_path, detection_model, epoch)
        

    print(f"Epoch: {epoch}\tLoss: {epoch_loss / len(dataloader_for_detection)}")

    checkpoint_filename = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': detection_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

    if det_logging:
        experiment.log_metric("loss", epoch_loss, step=epoch)
    
    # logging model weights (accorging to log_interval + last epoch)
    if det_logging and (epoch % log_interval == 0 or epoch == det_epochs-1):
        torch.save(detection_model, 'det_model.pth')
        experiment.log_model(name = f"model-epoch-{epoch}", file_or_folder = 'det_model.pth', file_name = f"det-model-epoch-{epoch}")
        experiment.log_asset(file_data = 'det_model.pth', file_name = f"det_model-epoch-{epoch}")
        print("save model")


if det_logging:
    experiment.end()

  0%|          | 0/1005 [00:00<?, ?it/s]

/Users/andreisuhov/Desktop/DL projects/Face_id_and_detection/src/models.py:298: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_class = F.mse_loss(pred_class, target_class)
  1%|          | 8/1005 [00:04<09:42,  1.71it/s]


KeyboardInterrupt: 

In [ ]:
#Face recognition setup
conv = ConvEmbedding().to(device)
triplet_model = Triplet(conv).to(device)
loss_for_recognition = TripletLoss(margin=5)
optimizer = torch.optim.Adam(triplet_model.parameters(), lr=1e-3)

In [ ]:
len(recognition_dataloader)

NameError: name 'recognition_dataloader' is not defined

In [ ]:
rec_epochs = config['recognition__epochs']
rec_logging = config['recognition_logging']
rec_log_interval = config["recognition_log_wieghts_interval"]

if rec_logging: 
    # read secrets for cometml logging
    with open('secrets.json') as secrets_file:
        secrets = json.load(secrets_file)

    # init experimenxt
    experiment = Experiment(
        api_key=secrets["api_key"],
        project_name=secrets["project_name"],
        workspace="reu-ds-club", 
        tags=["recognition"],
    )

    hyper_params = {
        "model_name": config["model"],
        "use_colab": config['use_colab'], 
        "epochs": rec_epochs,
        "batch_size": config['batch_size'], 
        "image_size": config['img_size'], 
    }

    experiment.log_parameters(hyper_params)

for epoch in range(rec_epochs):
    epoch_loss = 0

    for i, triplet in enumerate(recognition_dataloader):

        anc, pos, neg = triplet

        preds = triplet_model(anc.to(device), pos.to(device), neg.to(device))

        loss_val = loss_for_recognition(*preds)

        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        
        epoch_loss += loss_val.item()
        
        if i%5==0:
            print(f'{epoch} | [{i}/{len(recognition_dataloader)}] LOSS: {loss_val}')
        
    print(f'{epoch} | EPOCH LOSS: {loss_val}')

    torch.save({
        'epoch': epoch,
        'model_state_dict': triplet_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

    if logging:
        experiment.log_metric("loss", epoch_loss, step=epoch)
    
    # logging model weights (accorging to log_interval + last epoch)
    if logging and (epoch % log_interval == 0 or epoch == rec_epochs-1):
        torch.save(triplet_model, 'rec_model.pth')
        experiment.log_model(name = f"rec_model-epoch-{epoch}", file_or_folder = 'rec_model.pth', file_name = f"rec_model-epoch-{epoch}")
        experiment.log_asset(file_data = 'rec_model.pth', file_name = f"rec_model-epoch-{epoch}")
        print("save model")


if logging:
    experiment.end()

0 | [0/4083] LOSS: 4.9899678230285645
0 | [5/4083] LOSS: 4.828950881958008
0 | [10/4083] LOSS: 2.0965113639831543
0 | [15/4083] LOSS: 4.954373836517334
0 | [20/4083] LOSS: 4.998796463012695
0 | [25/4083] LOSS: 5.002302646636963
0 | [30/4083] LOSS: 4.9977312088012695
0 | [35/4083] LOSS: 4.992169380187988
0 | [40/4083] LOSS: 4.998960494995117
0 | [45/4083] LOSS: 4.997317790985107
0 | [50/4083] LOSS: 5.000369548797607
0 | [55/4083] LOSS: 5.000226020812988
0 | [60/4083] LOSS: 5.001129150390625
0 | [65/4083] LOSS: 4.996389865875244
0 | [70/4083] LOSS: 5.004185676574707
0 | [75/4083] LOSS: 5.024327278137207
0 | [80/4083] LOSS: 4.999454498291016
0 | [85/4083] LOSS: 4.999821186065674
0 | [90/4083] LOSS: 5.001814842224121
0 | [95/4083] LOSS: 4.934751033782959


KeyboardInterrupt: 